### Problem 1

##### Short Version

I've got a goofy little farming simulation I like. In order to maximize my profits, I want to calculate the compound return of crops where you buy seeds, harvest crops over a certain duration, sell those crops and reinvest the earnings into more seeds.

There are two types of crops (broadly) so I've written two subclasses to take in these different types of crops and calculate a compound calculation.

##### Long Version
I've decided to write a class to help me in a goofy little video game I like to play, Stardew Valley. Stardew Valley is a simple little "farming" simulation game in the vein of Harvest Moon. That's putting it simplistically... but this isn't Stardew Apologetics. Wikipedia can explain it better than me: https://en.wikipedia.org/wiki/Stardew_Valley

One of the primary features of this game is (obviously) farming. You can buy seeds for a variety of crops and plant them with various different growth rates and then sell them at different prices. Being the sort of perfectionist I am at these kinds of silly, resource building games I immediatley wanted to maximize my revenue.

Looking online, I immediately understood from the sites I was visiting that these authors really didn't understand the idea of compounding. When trying to find recommendations on crops during different seasons to focus on to make the most money, all of the articles I read evaluated crops in terms of "coins per day". Essentially, if something like (price-cost)/growth. This sounds right for most people, but I knew immediatley that a better model would be a compounding return model. Essentially, it relates to that old riddle you might have heard when you were a kid: "Would you rather have a million dollars a day, or have a penny double every day for a month". And if anyone does the map, despite that a penny doubling doesn't seem like it would get very far any time soon, you'll quickly realize how quickly it adds up, easily dwarfing a million dollars.

So naturally, I knew that it wasn't simple enough to evaluate crops from a "earnings per day" method when you can reinvest those earnings for duration of 28 days. So I wanted to calculate what that would be for every crop to make an informed decision on which crop to invest in. But, since this is a little made up world, I had to write very specific rules other than the standard compounding formula (which would be easy to write a function for).

Things like, for instance, if the shop is closed, you can't buy the crop to plant that day, so you need a way to calculate what the return will be if you have to reinvest every 5 days rather than every 4 days. Also, there are no fractions in this game. So you often will buy some number of seeds that's an integer and your return will be an integer. So something to track the remainder for every purchase and then reinvests the revenue and the remainder was necessary.

There are also two different types of crops that effect this class. Firstly, some crops have a set growth duration. Meaning than if you plant them on x day, they will yield a crop x + n days. These types of crops are what I call "nonrenewable" You plant, you harvest them, and then you must replant them with the proceeds from your harvest. However some crops, are what I call "renewable". They continue to produce on a cycle without having to be replanted. They always have long intial durations and for the first harvest, and then have shorter durations for every subsequent harvest. So the fist yield may take 10 days but subsequently they will continue to reyield crops without having to replant seeds.

In [1]:
class Crop:
    
    season_duration = 28
    
    def __init__(self, crop, cost, sell_price, growth_duration):
        self.crop = crop
        self.cost = cost
        self.sell_price = sell_price
        self.growth_duration = growth_duration
        
        self.leftover_days = Crop.season_duration % (self.growth_duration + 1)
        self.harvest_count = int((Crop.season_duration - self.leftover_days) / (self.growth_duration + 1))
    
    def get_crop(self):
        return self.crop
    
    def get_cost(self):
        return self.cost
    
    def get_sell_price(self):
        return self.sell_price
    
    def get_growth_duration(self):
        return self.growth_duration
            
    def shop_closed(self, start=0):
        end = Crop.season_duration
        
        return [x+1 for x in range(start-1,end) if (x+1) % 7 == 3]
    
    def pay_days(self, start=1):
        end = Crop.season_duration
        
        isharvest = lambda x: x % (self.growth_duration+1) == 0
        isclosed = lambda x: x % 7 == 3
        
        return [x+start-1 if isharvest(x) and not isclosed(x+1) else x+start for x in range(start,end+1) if isharvest(x)]
    
    def harvest(self):
        print(self.period)
        print(self.harvest_count)
        
class NonRenewCrop(Crop):
        
    def __init__(self, crop, cost, sell_price, growth_duration):
        super().__init__(crop, cost, sell_price, growth_duration)

        self.leftover_days = Crop.season_duration % self.growth_duration
        self.harvest_count = int((Crop.season_duration - self.leftover_days) / self.growth_duration + 1)

    def compound(self, account=500):
        for x in self.pay_days():
            remainder = account % self.cost
            revenue = ((account - (remainder))/self.cost)*self.sell_price
            account = revenue + remainder
            
        return account

class RenewCrop(Crop):
        
    def __init__(self, crop, cost, sell_price, growth_duration, intial_grow):
        super().__init__(crop, cost, sell_price, growth_duration)
        self.intial_grow = intial_grow

        self.leftover_days = Crop.season_duration % self.growth_duration
        self.harvest_count = int((Crop.season_duration - self.leftover_days) / self.growth_duration + 1)
        
    def compound(self, account=500):
        pass


In [2]:
crops =      [['Parsnip', 20, 35, 4], 
              ['Potato', 50, 100, 6], 
              ['Cauliflower', 80, 175, 12], 
              ['Kale', 70, 110, 6],
              ['Garlic', 40, 60, 4]]

In [3]:
for crop in crops:
    crop = NonRenewCrop(crop[0],crop[1],crop[2],crop[3])
    print(crop.crop, crop.compound())
    

Parsnip 8135.0
Potato 8000.0
Cauliflower 2305.0
Kale 2980.0
Garlic 3680.0


Despite the Internet's love of the Potato, from a compound perspective, Parsnips are better for an overall return since their growth_duration (therefore the reinvestment cycle) is shorter.

### Problem 2

Since the class started I was recommended to try Codewars (https://www.codewars.com/
) to practice my coding skills. People sbumit interesting algorthimic problems for people to sharpen their teeth on and both of my comprehensions are attempted solutions to those problems.

#### Premise 1
Write a function that takes in a string of one or more words, and returns the same string, but with all five or more letter words reversed (Just like the name of this Kata). Strings passed in will consist of only letters and spaces. Spaces will be included only when more than one word is present.

Examples: spinWords( "Hey fellow warriors" ) => returns "Hey wollef sroirraw" spinWords( "This is a test") => returns "This is a test" spinWords( "This is another test" )=> returns "This is rehtona test"

In [4]:
problems = ["Single word", "Welcome to CodeWars", "Multiple Words", "Hey fellow Warrors"]

#### For Loop Solution:

In [5]:
for problem in problems:
    sentence = problem
    solution = []
    for w in sentence.split():
        if len(w) >= 5:
            solution.append(w[::-1])
        else:
            solution.append(w)
        
    print(' '.join(solution))
    

elgniS word
emocleW to sraWedoC
elpitluM sdroW
Hey wollef srorraW


#### List Comp Solution

In [6]:
def spin_words(sentence):
    return ' '.join([w[::-1] if len(w)>=5 else w for w in sentence.split()])

for problem in problems:
    print(spin_words(problem))

elgniS word
emocleW to sraWedoC
elpitluM sdroW
Hey wollef srorraW


#### lambda Solution

In [7]:
for problem in problems:
    solution = ' '.join(list(map(lambda w: w[::-1] if len(w)>=5 else w, problem.split())))
    
    print(solution)

elgniS word
emocleW to sraWedoC
elpitluM sdroW
Hey wollef srorraW


#### Premise 2
Write an algorithm that takes an array and moves all of the zeros to the end, preserving the order of the other elements.

In [8]:
problems = [[1,2,0,1,0,1,0,3,0,1],
            [9,0.0,0,9,1,2,0,1,0,1,0.0,3,0,1,9,0,0,0,0,9], 
            ["a",0,0,"b","c","d",0,1,0,1,0,3,0,1,9,0,0,0,0,9], 
            ["a",0,0,"b",None,"c","d",0,1,False,0,1,0,3,[],0,1,9,0,0,{},0,0,9], 
            [0,1,None,2,False,1,0]
           ]

#### For Loop Solution:

In [9]:
for problem in problems:
    array = problem
    zeroes = []
    solution = []
    for item in array:
        if item !=0 or isinstance(item, bool):
            solution.append(item)
        else:
            zeroes.append(item)
    
    print(solution + zeroes)

[1, 2, 1, 1, 3, 1, 0, 0, 0, 0]
[9, 9, 1, 2, 1, 1, 3, 1, 9, 9, 0.0, 0, 0, 0, 0.0, 0, 0, 0, 0, 0]
['a', 'b', 'c', 'd', 1, 1, 3, 1, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['a', 'b', None, 'c', 'd', 1, False, 1, 3, [], 1, 9, {}, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, None, 2, False, 1, 0, 0]


#### List Comp Solution

In [10]:
def move_zeros(array):
    no_zeroes = [x for x in array if x != 0 or isinstance(x, bool)]

    solution = no_zeroes + (len(array) - len(no_zeroes))*[0]
    
    return solution

for problem in problems:
    print(move_zeros(problem))

[1, 2, 1, 1, 3, 1, 0, 0, 0, 0]
[9, 9, 1, 2, 1, 1, 3, 1, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['a', 'b', 'c', 'd', 1, 1, 3, 1, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['a', 'b', None, 'c', 'd', 1, False, 1, 3, [], 1, 9, {}, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, None, 2, False, 1, 0, 0]


#### lambda Solution

In [11]:
for problem in problems:
    no_zeroes = list(filter(lambda x: x!= 0 or isinstance(x,bool) ,problem)) 
    
    print(no_zeroes + (len(array) - len(no_zeroes))*[0])

[1, 2, 1, 1, 3, 1, 0]
[9, 9, 1, 2, 1, 1, 3, 1, 9, 9]
['a', 'b', 'c', 'd', 1, 1, 3, 1, 9, 9]
['a', 'b', None, 'c', 'd', 1, False, 1, 3, [], 1, 9, {}, 9]
[1, None, 2, False, 1, 0, 0]


### Problem 3

Another challenge from codewars was fairly difficult for me to solve. I'll outline the premise of the challenge below:

#### Premise 1

Some numbers have funny properties. For example:

89 --> 8¹ + 9² = 89 * 1

695 --> 6² + 9³ + 5⁴= 1390 = 695 * 2

46288 --> 4³ + 6⁴+ 2⁵ + 8⁶ + 8⁷ = 2360688 = 46288 * 51

Given a positive integer n written as abcd... (a, b, c, d... being digits) and a positive integer p

we want to find a positive integer k, if it exists, such as the sum of the digits of n taken to the successive powers of p is equal to k * n.
In other words:

Is there an integer k such as : (a ^ p + b ^ (p+1) + c ^(p+2) + d ^ (p+3) + ...) = n * k

If it is the case we will return k, if not return -1.

Note: n and p will always be given as strictly positive integers.

#### Solution Explanation and Comparison
Intially, when I began this tool I solved it by using a list comprehension. I essentially converted the integer into a string of numbers, conerted that string of numbers into a list, then ran the enumerate built-in starting at p to raise each iteration of the number in the string of numbers to the power of p^n

In [12]:
import timeit

mysetup = ''
mycode = '''
def dig_pow(n,p):
    
    nk = sum([(int(x)**i) for i,x in enumerate(list(str(n)),p)])

    if nk % n == 0:
        return (int(nk/n))
    return - 1
'''
list_comp_solution = timeit.repeat(setup=mysetup, stmt=mycode, repeat=10000, number=1000000)

It then occurred to me after my submission, that this might be an ideal place for a generator. Rather than summing a list comprehension it might be more efficient to simply sum a generator.

So I made two changes, I changed the list comprehension into a generator by simply changing the outside sqare brackets into (). ThenI also got rid of the list-built in and simply enumerated across the str(n) without convering it into a list.

In [13]:
mysetup = ''
mycode = '''
def dig_pow(n,p):
    
    nk = sum(((int(x)**i) for i,x in enumerate(str(n),p)))

    if nk % n == 0:
        return (int(nk/n))
    return - 1
'''
generator_solution = timeit.repeat(setup=mysetup, stmt=mycode,repeat=10000,number=1000000)

In [14]:
import statistics

print(f'For the Generator the average run time was: {statistics.mean(generator_solution):.4f} the STDV was: {statistics.stdev(generator_solution):.4f}')
print(f'For the List comprehension solution the average run time was: {statistics.mean(list_comp_solution):.4f} the STDV was: {statistics.stdev(list_comp_solution):.4f}')

For the Generator the average run time was: 0.0567 the STDV was: 0.0134
For the List comprehension solution the average run time was: 0.0580 the STDV was: 0.0149


I ran these two solutions through the timeit module with many iterations and it appears that the generator solution to the dig_pow() function is a bit faster.